# 📦 Step 1: Import Libraries and Utils

In [ ]:
import sys
import os
import shutil
import pickle

sys.path.insert(1, './code/utils')
from pocketvec_utils import (
    create_parameter_file,
    environmental_variables,
    create_cavity,
    run_rDock,
    create_file_scores,
    read_rDock_scores,
    raw_fp,
    rank_fp
)

## 🧩 Step 2: Define Inputs

In [ ]:
receptor = "run_pocketvec/1A42_prepared.mol2"
pocket_centroid = "run_pocketvec/CTR_LIG.sd"
outpath = "run_pocketvec"
lib = "data/libs/TOP_128_rDock_LLM.sdf"
n_runs = 25
radius = 12
seed = 42
rDock = "code/utils/rDock_compiled/"

## 🛠️ Step 3: Setup Parameters

In [ ]:
os.makedirs(outpath, exist_ok=True)
param_file = os.path.join(outpath, "st_parameters.prm")
create_parameter_file(param_file, receptor, pocket_centroid, radius=radius)
environmental_variables(rDock)

## 📍 Step 4: Generate Cavity

In [ ]:
log_path = os.path.join(outpath, "cavity_log.log")
grid_path = os.path.join(outpath, "cavity.grd")
create_cavity(outpath, log_path, grid_path, rbcavity="rbcavity")

## 📎 Step 5: Copy Docking Parameter File

In [ ]:
shutil.copyfile("code/utils/rDock_compiled/dock.prm", os.path.join(outpath, "dock.prm"))

## 🚀 Step 6: Run Docking

In [ ]:
run_rDock(outpath, lib, nruns=n_runs, seed=seed)

## 📊 Step 7: Create Scores and Fingerprint

In [ ]:
scores_file = os.path.join(outpath, "scores.tsv")
results_file = os.path.join(outpath, "results.sd")
create_file_scores(scores_file, results_file)

sorted_lib = 'data/libs/order/' + os.path.basename(lib).replace(".sdf", ".pkl")
scores = read_rDock_scores(scores_file)
raw = raw_fp(scores, sorted_lib)
rank = rank_fp(raw)

pickle.dump(rank, open(os.path.join(outpath, "PocketVec_fp.pkl"), "wb"))

## ✅ Done

In [ ]:
print("✅ PocketVec pipeline completed successfully!")